**SPEECH EMOTION DETECTION USING MLP**

> Indented block



In [4]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}


In [5]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [6]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          y.append(emotion)
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [7]:
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score

import os
import pickle

In [8]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("[+] Number of features:", X_train.shape[1])
# best model, determined by a grid search
model_params = {
    'alpha': 0.1,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (256,), 
    'learning_rate': 'adaptive', 
    'max_iter': 600, 
    'solver' : 'adam',
    'verbose' : True
}
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
model = MLPClassifier(**model_params)

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

# now we save the model
# make result directory if doesn't exist yet
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))

03-01-01-01-01-01-01.wav
03-01-01-01-01-02-01.wav
03-01-01-01-02-01-01.wav
03-01-01-01-02-02-01.wav
03-01-02-01-01-01-01.wav
03-01-02-01-01-02-01.wav
03-01-02-01-02-01-01.wav
03-01-02-01-02-02-01.wav
03-01-02-02-01-01-01.wav
03-01-02-02-01-02-01.wav
03-01-02-02-02-01-01.wav
03-01-02-02-02-02-01.wav
03-01-03-01-01-01-01.wav
03-01-03-01-01-02-01.wav


C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[ 3.0517578e-05  3.0517578e-05  3.0517578e-05 ...  0.0000000e+00
  0.0000000e+00 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[3.0517578e-05 3.0517578e-05 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

03-01-03-01-02-01-01.wav
03-01-03-01-02-02-01.wav
03-01-03-02-01-01-01.wav
03-01-03-02-01-02-01.wav
03-01-03-02-02-01-01.wav
03-01-03-02-02-02-01.wav
03-01-04-01-01-01-01.wav


C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.0517578e-05 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


03-01-04-01-01-02-01.wav
03-01-04-01-02-01-01.wav
03-01-04-01-02-02-01.wav
03-01-04-02-01-01-01.wav
03-01-04-02-01-02-01.wav
03-01-04-02-02-01-01.wav
03-01-04-02-02-02-01.wav
03-01-05-01-01-01-01.wav
03-01-05-01-01-02-01.wav
03-01-05-01-02-01-01.wav
03-01-05-01-02-02-01.wav
03-01-05-02-01-01-01.wav


...

03-01-01-01-01-02-23.wav
03-01-01-01-02-01-23.wav
03-01-01-01-02-02-23.wav
03-01-02-01-01-01-23.wav
03-01-02-01-01-02-23.wav
03-01-02-01-02-01-23.wav
03-01-02-01-02-02-23.wav
03-01-02-02-01-01-23.wav
03-01-02-02-01-02-23.wav
03-01-02-02-02-01-23.wav
03-01-02-02-02-02-23.wav
03-01-03-01-01-01-23.wav
03-01-03-01-01-02-23.wav
03-01-03-01-02-01-23.wav
03-01-03-01-02-02-23.wav
03-01-03-02-01-01-23.wav
03-01-03-02-01-02-23.wav
03-01-03-02-02-01-23.wav
03-01-03-02-02-02-23.wav
03-01-04-01-01-01-23.wav
03-01-04-01-01-02-23.wav
03-01-04-01-02-01-23.wav
03-01-04-01-02-02-23.wav
03-01-04-02-01-01-23.wav
03-01-04-02-01-02-23.wav
03-01-04-02-02-01-23.wav
03-01-04-02-02-02-23.wav
03-01-05-01-01-01-23.wav
03-01-05-01-01-02-23.wav
03-01-05-01-02-01-23.wav
03-01-05-01-02-02-23.wav
03-01-05-02-01-01-23.wav
03-01-05-02-01-02-23.wav
03-01-05-02-02-01-23.wav
03-01-05-02-02-02-23.wav
03-01-06-01-01-01-23.wav
03-01-06-01-01-02-23.wav
03-01-06-01-02-01-23.wav
03-01-06-01-02-02-23.wav
03-01-06-02-01-01-23.wav


C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[-3.0517578e-05 -3.0517578e-05  0.0000000e+00 ...  3.0517578e-05
  3.0517578e-05  3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


03-01-01-01-02-01-24.wav
03-01-01-01-02-02-24.wav
03-01-02-01-01-01-24.wav
03-01-02-01-01-02-24.wav
03-01-02-01-02-01-24.wav
03-01-02-01-02-02-24.wav
03-01-02-02-01-01-24.wav
03-01-02-02-01-02-24.wav
03-01-02-02-02-01-24.wav
03-01-02-02-02-02-24.wav
03-01-03-01-01-01-24.wav
03-01-03-01-01-02-24.wav
03-01-03-01-02-01-24.wav
03-01-03-01-02-02-24.wav
03-01-03-02-01-01-24.wav
03-01-03-02-01-02-24.wav
03-01-03-02-02-01-24.wav
03-01-03-02-02-02-24.wav
03-01-04-01-01-01-24.wav
03-01-04-01-01-02-24.wav
03-01-04-01-02-01-24.wav
03-01-04-01-02-02-24.wav
03-01-04-02-01-01-24.wav
03-01-04-02-01-02-24.wav
03-01-04-02-02-01-24.wav
03-01-04-02-02-02-24.wav


C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.5258789e-04
 -1.5258789e-04 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


03-01-05-01-01-01-24.wav
03-01-05-01-01-02-24.wav
03-01-05-01-02-01-24.wav
03-01-05-01-02-02-24.wav
03-01-05-02-01-01-24.wav
03-01-05-02-01-02-24.wav
03-01-05-02-02-01-24.wav
03-01-05-02-02-02-24.wav
03-01-06-01-01-01-24.wav
03-01-06-01-01-02-24.wav
03-01-06-01-02-01-24.wav
03-01-06-01-02-02-24.wav
03-01-06-02-01-01-24.wav
03-01-06-02-01-02-24.wav
03-01-06-02-02-01-24.wav
03-01-06-02-02-02-24.wav
03-01-07-01-01-01-24.wav
03-01-07-01-01-02-24.wav
03-01-07-01-02-01-24.wav
03-01-07-01-02-02-24.wav
03-01-07-02-01-01-24.wav
03-01-07-02-01-02-24.wav
03-01-07-02-02-01-24.wav
03-01-07-02-02-02-24.wav
03-01-08-01-01-01-24.wav
03-01-08-01-01-02-24.wav
03-01-08-01-02-01-24.wav
03-01-08-01-02-02-24.wav
03-01-08-02-01-01-24.wav
03-01-08-02-01-02-24.wav
03-01-08-02-02-01-24.wav
03-01-08-02-02-02-24.wav
[+] Number of training samples: 504
[+] Number of testing samples: 168
[+] Number of features: 180
[*] Training the model...
Iteration 1, loss = 24.63363520
Iteration 2, loss = 21.94200944
Iteration 3

In [9]:
model = MLPClassifier(**model_params)

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

[*] Training the model...
Iteration 1, loss = 24.14644812
Iteration 2, loss = 17.50795471
Iteration 3, loss = 8.89451520
Iteration 4, loss = 8.40281611
Iteration 5, loss = 10.74270226
Iteration 6, loss = 8.36254323
Iteration 7, loss = 6.13310605
Iteration 8, loss = 6.97901389
Iteration 9, loss = 5.77260979
Iteration 10, loss = 3.10651028
Iteration 11, loss = 2.71071376
Iteration 12, loss = 4.00679847
Iteration 13, loss = 2.79241530
Iteration 14, loss = 1.83786904
Iteration 15, loss = 2.56731031
Iteration 16, loss = 2.53817025
Iteration 17, loss = 1.71443864
Iteration 18, loss = 1.65919086
Iteration 19, loss = 1.83244945
Iteration 20, loss = 1.36811628
Iteration 21, loss = 1.45443861
Iteration 22, loss = 1.41712642
Iteration 23, loss = 1.21964110
Iteration 24, loss = 1.24029299
Iteration 25, loss = 1.17875973
Iteration 26, loss = 1.11051360
Iteration 27, loss = 1.08058031
Iteration 28, loss = 1.05732989
Iteration 29, loss = 1.00087359
Iteration 30, loss = 1.01738486
Iteration 31, loss =

Iteration 252, loss = 0.26634839
Iteration 253, loss = 0.26805284
Iteration 254, loss = 0.27154335
Iteration 255, loss = 0.26689343
Iteration 256, loss = 0.26337741
Iteration 257, loss = 0.25780299
Iteration 258, loss = 0.26461956
Iteration 259, loss = 0.26096323
Iteration 260, loss = 0.25706906
Iteration 261, loss = 0.25440836
Iteration 262, loss = 0.25016087
Iteration 263, loss = 0.25219284
Iteration 264, loss = 0.25461511
Iteration 265, loss = 0.25513805
Iteration 266, loss = 0.25391049
Iteration 267, loss = 0.25344983
Iteration 268, loss = 0.24661647
Iteration 269, loss = 0.25236466
Iteration 270, loss = 0.25429435
Iteration 271, loss = 0.24456433
Iteration 272, loss = 0.25020160
Iteration 273, loss = 0.25067372
Iteration 274, loss = 0.24728298
Iteration 275, loss = 0.24153502
Iteration 276, loss = 0.24528307
Iteration 277, loss = 0.23761910
Iteration 278, loss = 0.23837746
Iteration 279, loss = 0.24415749
Iteration 280, loss = 0.24070985
Iteration 281, loss = 0.23812658
Iteration 

In [10]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.86      0.89      0.87        61
       happy       0.75      0.73      0.74        41
     neutral       0.64      0.67      0.65        21
         sad       0.84      0.80      0.82        45

    accuracy                           0.80       168
   macro avg       0.77      0.77      0.77       168
weighted avg       0.80      0.80      0.80       168

[[54  4  2  1]
 [ 6 30  3  2]
 [ 1  2 14  4]
 [ 2  4  3 36]]


In [11]:
import os
import wave
import pickle
from sys import byteorder
from array import array
from struct import pack
from sklearn.neural_network import MLPClassifier


In [14]:
#print("Please talk")
filename = "speech-emotion-recognition-ravdess-data/Actor_01/03-01-02-02-01-02-01.wav"# record the file (start talking)
    #record_to_file(filename)
    # extract features and reshape it
features = extract_feature(filename, mfcc=True, chroma=True, mel=True).reshape(1, -1)
    # predict
result = model.predict(features)[0]
    # show the result !
print("result:", result)

result: neutral


C:\Users\MANAL\AppData\Local\Temp\ipykernel_27804\1163463125.py:31: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
